In [2]:
import pandas as pd
import numpy as np
from scipy.stats import trim_mean
from statsmodels import robust
import wquantiles

import seaborn as sns
import matplotlib.pylab as plt

%matplotlib inline

In [3]:
# Read data
booking = pd.read_csv('/Users/joaopedro/Documents/MSBA/Classes/BAX 452 - Machine Learning/Assignments/Final Project - Booking Price Pred/booking_hotels.csv')
booking.head()

,_id,name,city,room_type,free_breakfast,free_cancellation,review,star_rating,total_price,price_per_day,preferred_partner_property,city_population,lat,lon
0,1,GreenTree Pasadena,Pasadena,King Room,1,0,1586,3,1322,165.25,1,12750807,34.138934,-118.146914
1,2,Big Bear Inn,Big Bear Lake,Standard King Room,0,0,720,2,909,113.62,0,12750807,34.243504,-116.915642
2,3,Red Lion Inn & Suites Auburn,Auburn,King Room - Non-Smoking,1,0,667,3,803,100.38,1,3592294,38.898076,-121.071247
3,4,Regency Inn in Los Angeles,"Northeast Los Angeles, Los Angeles",King Room (Non Smoking Only),0,0,313,2,975,121.88,1,12750807,34.139681,-118.218387
4,5,Hotel La Rose,Santa Rosa,Junior Suite,0,1,634,3,1642,205.25,0,37708,38.437927,-122.720924


In [5]:
# Info about the columns
booking.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 14 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   _id                         1000 non-null   int64  
 1   name                        1000 non-null   object 
 2   city                        1000 non-null   object 
 3   room_type                   1000 non-null   object 
 4   free_breakfast              1000 non-null   int64  
 5   free_cancellation           1000 non-null   int64  
 6   review                      1000 non-null   int64  
 7   star_rating                 1000 non-null   int64  
 8   total_price                 1000 non-null   int64  
 9   price_per_day               1000 non-null   float64
 10  preferred_partner_property  1000 non-null   int64  
 11  city_population             1000 non-null   int64  
 12  lat                         1000 non-null   float64
 13  lon                         1000 n

In [4]:
# Describing the data
booking.describe()

,_id,free_breakfast,free_cancellation,review,star_rating,total_price,price_per_day,preferred_partner_property,city_population,lat,lon
count,1000.000000,1000.00000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1.000000e+03,1000.000000,1000.000000
mean,500.500000,0.41600,0.542000,662.292000,2.836000,1439.888000,179.985900,0.889000,2.143128e+06,35.731320,-119.349634
std,288.819436,0.49314,0.498482,663.642514,0.840125,870.407064,108.800915,0.314289,4.063794e+06,2.301312,3.668274
min,1.000000,0.00000,0.000000,0.000000,0.000000,360.000000,45.000000,0.000000,3.770800e+04,25.924894,-124.199322
25%,250.750000,0.00000,0.000000,250.750000,2.000000,965.750000,120.717500,1.000000,3.770800e+04,33.909259,-121.905275
50%,500.500000,0.00000,1.000000,457.500000,3.000000,1253.500000,156.685000,1.000000,3.770800e+04,34.828950,-118.851827
75%,750.250000,1.00000,1.000000,849.000000,3.000000,1673.250000,209.152500,1.000000,3.592294e+06,37.671757,-117.866005
max,1000.000000,1.00000,1.000000,5127.000000,5.000000,11510.000000,1438.750000,1.000000,2.099600e+07,47.008629,-78.835789


In [6]:
booking['city'].value_counts()

Union Square, San Francisco      25
San Jose                         23
Downtown San Diego, San Diego    21
LAX Area, Los Angeles            17
Hollywood, Los Angeles           16
                                 ..
Bell                              1
Westley                           1
Hollister                         1
Shelter Cove                      1
Tulare                            1
Name: city, Length: 354, dtype: int64

In [58]:
# Spliting location name to get city
df = pd.DataFrame()
df[['location', 'city_n']] = booking['city'].str.split(',', 1, expand=True)
df.head(5)

,location,city_n
0,Pasadena,None
1,Big Bear Lake,None
2,Auburn,None
3,Northeast Los Angeles,Los Angeles
4,Santa Rosa,None


In [74]:
# Create a new column with the logic:
# For rows where city_n = None, get location, else get city_n
df['city_n2'] = np.where(~df['city_n'].isnull(),df['city_n'],df['location'])

for i in df['city_n2']:
    i.lstrip()

# Removing whitespaces from beginning of string
df['city_n2'] = [i.lstrip() for i in df['city_n2']]

# Checking the values
df['city_n2'].value_counts().head(5)

Los Angeles      95
San Diego        69
San Francisco    45
San Jose         29
Anaheim          20
Name: city_n2, dtype: int64

In [77]:
# Putting the clean city column back into the original dataframe
booking['city_new'] = df['city_n2']
booking.head()

,_id,name,city,room_type,free_breakfast,free_cancellation,review,star_rating,total_price,price_per_day,preferred_partner_property,city_population,lat,lon,city_new
0,1,GreenTree Pasadena,Pasadena,King Room,1,0,1586,3,1322,165.25,1,12750807,34.138934,-118.146914,Pasadena
1,2,Big Bear Inn,Big Bear Lake,Standard King Room,0,0,720,2,909,113.62,0,12750807,34.243504,-116.915642,Big Bear Lake
2,3,Red Lion Inn & Suites Auburn,Auburn,King Room - Non-Smoking,1,0,667,3,803,100.38,1,3592294,38.898076,-121.071247,Auburn
3,4,Regency Inn in Los Angeles,"Northeast Los Angeles, Los Angeles",King Room (Non Smoking Only),0,0,313,2,975,121.88,1,12750807,34.139681,-118.218387,Los Angeles
4,5,Hotel La Rose,Santa Rosa,Junior Suite,0,1,634,3,1642,205.25,0,37708,38.437927,-122.720924,Santa Rosa


The next step is to group different room types. Based on our dataset, we gonna group the rooms in 3 types:
1. King - Rooms that have the string king in the room type
2. Queen - Rooms that have the string queen in the room type
3. Other - Other occurences

In [128]:
booking['room_type'].value_counts().head()

King Room                                       94
King Room - Non-Smoking                         80
Queen Room with Two Queen Beds                  50
Queen Room                                      46
Queen Room with Two Queen Beds - Non-Smoking    44
Name: room_type, dtype: int64

In [131]:
room = pd.DataFrame()
classification = []

for i in booking['room_type']:
    if 'King' in i:
        classification.append('King Room')
    elif 'Queen' in i:
        classification.append('Queen Room')
    else:
        classification.append('Other')

room['c'] = classification
room.value_counts()

# Putting the room classification column back in the df
booking['room_classification'] = room['c']

# Checking the result
booking.head()

,_id,name,city,room_type,free_breakfast,free_cancellation,review,star_rating,total_price,price_per_day,preferred_partner_property,city_population,lat,lon,city_new,room_classification
0,1,GreenTree Pasadena,Pasadena,King Room,1,0,1586,3,1322,165.25,1,12750807,34.138934,-118.146914,Pasadena,King Room
1,2,Big Bear Inn,Big Bear Lake,Standard King Room,0,0,720,2,909,113.62,0,12750807,34.243504,-116.915642,Big Bear Lake,King Room
2,3,Red Lion Inn & Suites Auburn,Auburn,King Room - Non-Smoking,1,0,667,3,803,100.38,1,3592294,38.898076,-121.071247,Auburn,King Room
3,4,Regency Inn in Los Angeles,"Northeast Los Angeles, Los Angeles",King Room (Non Smoking Only),0,0,313,2,975,121.88,1,12750807,34.139681,-118.218387,Los Angeles,King Room
4,5,Hotel La Rose,Santa Rosa,Junior Suite,0,1,634,3,1642,205.25,0,37708,38.437927,-122.720924,Santa Rosa,Other


In [137]:
# Getting the dataframe with important columns
columns = ['name','city_new','room_classification','free_breakfast',
           'free_cancellation','review','star_rating','preferred_partner_property',
           'city_population','lat', 'lon']

booking_df = booking[columns]
booking_df.head()

,name,city_new,room_classification,free_breakfast,free_cancellation,review,star_rating,preferred_partner_property,city_population,lat,lon
0,GreenTree Pasadena,Pasadena,King Room,1,0,1586,3,1,12750807,34.138934,-118.146914
1,Big Bear Inn,Big Bear Lake,King Room,0,0,720,2,0,12750807,34.243504,-116.915642
2,Red Lion Inn & Suites Auburn,Auburn,King Room,1,0,667,3,1,3592294,38.898076,-121.071247
3,Regency Inn in Los Angeles,Los Angeles,King Room,0,0,313,2,1,12750807,34.139681,-118.218387
4,Hotel La Rose,Santa Rosa,Other,0,1,634,3,0,37708,38.437927,-122.720924


In [138]:
# Puting results in an excel file
booking_df.to_excel("/Users/joaopedro/Documents/MSBA/Classes/BAX 452 - Machine Learning/Assignments/Final Project - Booking Price Pred/booking.xlsx") 